#### Default Privacy
By default, `plotly.iplot()` and `plotly.plot()` create public graphs (which are free to create). With a [plotly subscription](https://plot.ly/plans) you can easily make charts private or secret via the sharing argument.

#### Public Graphs

In [1]:
import chart_studio.plotly as py
import plotly.graph_objects as go

data = [
    go.Scatter(
        x=[1, 2, 3],
        y=[1, 3, 1]
    )
]

py.iplot(data, filename='privacy-public', sharing='public')

Below is the URL of this public plot.  Anyone can view public plots even if they are not logged into Plotly.  Go ahead and try it out:

In [2]:
py.plot(data, filename='privacy-public', sharing='public')

'https://plot.ly/~PythonPlotBot/2677/'

### Private Graphs

In [3]:
py.iplot(data, filename='privacy-private', sharing='private')

Below is the URL of the private plot above. Only the owner can view the private plot. You won't be able to view this plot, try it out:

In [4]:
py.plot(data, filename='privacy-private', sharing='private')

'https://plot.ly/~PythonPlotBot/2679/'

### Secret Graphs

In [5]:
py.iplot(data, filename='privacy-secret', sharing='secret')

Below is the URL of this secret plot. Anyone with the secret link can view this chart. However, it will not appear in the Plotly feed, your profile, or search engines. Go ahead and try it out:

In [6]:
py.plot(data, filename='privacy-secret', sharing='secret')

'https://plot.ly/~PythonPlotBot/475?share_key=UaGz0FTFLklnEd7XTKaqy8'

### Make All Future Plots Private
To make all future plots private, you can update your configuration file to create private plots by default:

In [7]:
import chart_studio
chart_studio.tools.set_config_file(world_readable=False, sharing='private')

### Make All Existing Plots Private
This example uses [Plotly's REST API](https://api.plot.ly/v2/)

In [8]:
import json
import requests
from requests.auth import HTTPBasicAuth

Define variables, including YOUR [USERNAME and API KEY](https://plot.ly/settings/api)

In [9]:
username = 'private_plotly' # Replace with YOUR USERNAME
api_key = 'k0yy0ztssk' # Replace with YOUR API KEY

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}

page_size = 500

Collect filenames of <b>ALL</b> of your plots and <br>update `world_readable` of each plot with a PATCH request

In [ ]:
def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&filetype=plot&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content.decode('utf-8'))
    yield page
    while True:
        resource = page['children']['next']
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content.decode('utf-8'))
        yield page

def make_all_plots_private(username, page_size=500):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            requests.patch('https://api.plot.ly/v2/files/'+fid, {"world_readable": False}, auth=auth, headers=headers)
    print('ALL of your plots are now private - visit: https://plot.ly/organize/home to view your private plots!')

make_all_plots_private(username)

### Reference

In [10]:
help(py.plot)

Help on function plot in module chart_studio.plotly.plotly:

plot(figure_or_data, validate=True, **plot_options)
    Create a unique url for this plot in Plotly and optionally open url.
    
    plot_options keyword arguments:
    filename (string) -- the name that will be associated with this figure
    auto_open (default=True) -- Toggle browser options
        True: open this plot in a new browser tab
        False: do not open plot in the browser, but do return the unique url
    sharing ('public' | 'private' | 'secret') -- Toggle who can view this
                                                  graph
        - 'public': Anyone can view this graph. It will appear in your profile
                    and can appear in search engines. You do not need to be
                    logged in to Plotly to view this chart.
        - 'private': Only you can view this plot. It will not appear in the
                     Plotly feed, your profile, or search engines. You must be
                